### LazyMIL: Automated MIL Benchmarking and Smart Consensus Modeling

The **LazyMIL** module provides a convenient, high-level interface for applying **Multiple Instance Learning (MIL)** to real-world or benchmark datasets.  
It seamlessly combines **descriptor calculation**, **model training**, and **evaluation** into one streamlined workflow — ideal for competitions, benchmarks, or quick exploratory studies.

LazyMIL automatically:
- Handles **descriptor calculation** for molecules or fragments.  
- Trains **multiple MIL estimators** in parallel.  
- Collects predictions and metrics for model comparison.  
- Optionally integrates **smart consensus optimization** using a genetic algorithm.

For consensus modeling, LazyMIL leverages the **QSARcons** package — a flexible framework for discovering optimal model ensembles.

> 🧩 **Install QSARcons before running this tutorial:**
> ```bash
> pip install qsarcons
> ```

**In summary:**  
LazyMIL simplifies the process of testing, comparing, and combining MIL models, making it a practical tool for QSAR researchers and ML competitions alike.

In [1]:
import polaris
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from qsarmil.lazy import LazyMIL
from qsarcons.consensus import RandomSearchRegressor, SystematicSearchRegressor, GeneticSearchRegressor

### 1. Loading a Polaris Benchmark Dataset

In this example, we use one of the **Polaris** benchmark datasets — a curated collection of molecular property prediction tasks.  
Specifically, we load **`adme-fang-solu-1`**, a dataset focused on **aqueous solubility (LOG_SOLUBILITY)** prediction.

The benchmark provides pre-defined training and testing splits to ensure reproducibility and fair comparison across models.

In [2]:
# Load the benchmark from the Hub
benchmark = polaris.load_benchmark("polaris/adme-fang-solu-1")

# Get the train and test data-loaders
data_train, data_test = benchmark.get_train_test_split()
data_train, data_test = data_train.as_dataframe(), data_test.as_dataframe()

smi_train, prop_train = data_train["smiles"].to_list(), data_train["LOG_SOLUBILITY"].to_list()

data_train, data_val = train_test_split(data_train, test_size=0.2, random_state=42)

Fetching benchmark ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/1 0:00:02

            Success: Fetching benchmark                                                               ]8;id=167414;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=732052;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/utils/context.py#53\53]8;;\

### 2. Build multiple MIL models models with conformers as instances

In [3]:
data_test["LogS"] = [0 for i in data_test.index]

lazy_mil = LazyMIL(task="regression", hopt=False, output_folder="logs_bench", n_cpu=20, verbose=True)
lazy_mil.run(data_train, data_val, data_test)

Calculating descriptors: 100%|████████████████████████████████████████████████████████| 400/400 [01:40<00:00,  3.97it/s]


72 / 72 — MolFeatPmapper|DynamicPoolingNetworkRegressorsoressoressor


### 3. Build model consensus

In [4]:
metric = "auto"
cons_size = "auto"

In [5]:
cons_methods = [
    ("Best", SystematicSearchRegressor(cons_size=1, metric=metric)),         
    ("Random", RandomSearchRegressor(cons_size=cons_size, n_iter=1000, metric=metric)),       
    ("Systematic", SystematicSearchRegressor(cons_size=cons_size, metric=metric)),
    ("Genetic", GeneticSearchRegressor(cons_size=cons_size, n_iter=50, pop_size=50, mut_prob=0.2, metric=metric))
]

In [6]:
# load model predictions
df_val = pd.read_csv("logs_bench/val.csv")
df_test = pd.read_csv("logs_bench/test.csv")

# skip first two columns (smiles and true property value)
x_val, true_val = df_val.iloc[:, 2:], df_val.iloc[:, 1]
x_test = df_test.iloc[:, 2:]

In [7]:
for name, cons_searcher in cons_methods:

    # run search
    best_cons = cons_searcher.run(x_val, true_val)
    
    # make val and test predictions
    pred_val = cons_searcher._consensus_predict(x_val[best_cons])
    pred_test = cons_searcher._consensus_predict(x_test[best_cons])
    
    # write prediction accuracy metric
    df_val[name] = pred_val
    df_test[name] = pred_test

### 4. Summurize results

In [8]:
res = pd.DataFrame()
for model in df_val.columns[2:]:
    res.loc[model, "R2"] = r2_score(df_val["Y_TRUE"], df_val[model])

In [9]:
res.sort_values(by="R2", ascending=False)

,R2
Genetic,0.396709
Random,0.384397
Systematic,0.372215
RDKitGETAWAY|DynamicPoolingNetworkRegressor,0.311626
Best,0.311626
...,...
MolFeatUSRD|DynamicPoolingNetworkRegressor,0.064956
RDKitWHIM|MeanInstanceWrapperMLPNetworkRegressor,0.010812
MolFeatUSRD|MeanInstanceWrapperMLPNetworkRegressor,-0.015897
MolFeatPmapper|MeanBagWrapperMLPNetworkRegressor,-0.027218


In [10]:
y_pred = df_test["Genetic"].to_list()
results = benchmark.evaluate(y_pred)
results

{
  "benchmark_artifact_id": "polaris/adme-fang-solu-1",
  "benchmark_name": null,
  "benchmark_owner": null,
  "results": [
    {
      "test_set": "test",
      "target_label": "LOG_SOLUBILITY",
      "scores": {
        "pearsonr": 0.5320304922182691,
        "mean_squared_error": 0.39061086253604754,
        "r2": 0.27955320681794393,
        "mean_absolute_error": 0.4336216957394166,
        "spearmanr": 0.413671670214811,
        "explained_var": 0.28278434710607636
      }
    }
  ],
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "0.13.0",
  "slug": null,
  "github_url": null,
  "paper_url": null,
  "contributors": [],
  "artifact_id": null,
  "urn": null,
  "version": 1
}